In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torchvision
from torchvision.datasets import MNIST
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
dataset = MNIST(root='data/', download=True)
len(dataset)

In [4]:
test_dataset = MNIST(root='data/', train=False)
len(test_dataset)

In [5]:
dataset[0]

In [6]:
image, label = dataset[0]
plt.imshow(image, cmap='gray')
print('label:', label)

In [7]:
image, label = dataset[10]
plt.imshow(image, cmap='gray')
print('label:', label)

In [8]:
#transforming the PIL image into Tensors
dataset = MNIST(root='data/', 
                train=True, 
                transform=transforms.ToTensor())

In [9]:
img_tensor, label = dataset[0]
print(img_tensor.shape, label)

In [10]:
print(img_tensor[:,10:15,10:15])
print(torch.max(img_tensor), torch.min(img_tensor))

In [11]:
plt.imshow(img_tensor[0, 10:15, 10:15], cmap='gray')

In [12]:
def split_indices(n, val_pct):
    n_val= int(val_pct*n)
    idxs = np.random.permutation(n)
    return idxs[n_val:], idxs[:n_val]

In [13]:
train_indices, val_indices = split_indices(len(dataset), val_pct=0.2)

In [14]:
print(len(train_indices), len(val_indices))
print('Sample val indices: ', val_indices[:20])

In [34]:
import torch.nn as nn

from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader

import torch.nn.functional as F



In [16]:
#creating dataloader
batch_size = 100

#training sample and data loader
train_sampler = SubsetRandomSampler(train_indices)
train_loader = DataLoader(dataset, batch_size, sampler=train_sampler)

#validation sampler and data loader
val_sampler = SubsetRandomSampler(val_indices)
val_loader = DataLoader(dataset, batch_size, sampler=val_sampler)

In [17]:
input_size = 28*28
num_classes = 10

model = nn.Linear(input_size, num_classes)

In [18]:
print(model.weight.shape)
model.weight

In [19]:
print(model.bias.shape)
model.bias

In [20]:
for images, labels in train_loader:
    print(labels)
    print(images.shape)
    outputs = model(images)
    print(outputs)
    break

In [21]:
images.shape

In [29]:
class MnistModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, num_classes)
    
    def forward(self, xb):
        xb = xb.reshape(-1, 784)
        out = self.linear(xb)
        return out
    
model = MnistModel()

In [30]:
model.linear

In [31]:
print(model.linear.weight.shape, model.linear.bias.shape)
list(model.parameters())


In [32]:
for images, labels in train_loader:
    print(images.shape)
    outputs = model(images)
    break

print('outputs.shape : ', outputs.shape)
print('Sample outputs :\n', outputs[:2].data)

In [35]:
outputs[:2]

In [36]:
# Apply softmax for each output row
probs = F.softmax(outputs, dim=1)

# Look at sample probabilities
print("Sample probabilities:\n", probs[:2].data)

# Add up the probabilities of an output row
print("Sum: ", torch.sum(probs[0]).item())

In [37]:
max_probs, preds = torch.max(probs, dim=1)
print(preds)
print(max_probs)

In [38]:
labels